In [1]:
# Step 1: Install dependencies
!pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone \
unstructured pdfplumber pillow_heif unstructured_inference \
sentence-transformers pdfminer.six youtube-transcript-api pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [2]:
# Step 2: Import required libraries
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
from pinecone import Pinecone
import tiktoken
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata

In [3]:
os.environ['PINECONE_API_KEY'] = "pcsk_6hMzN_AKyGuU6fNsYeh9p6Hm523NE476hBhTApCBWjqkP3yNUMfURAMUWgAJLs3nnnB37"
os.environ['OPENROUTER_API_KEY'] = "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"

In [4]:
# Step 4: Initialize embedding and Pinecone
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
openrouter_client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.environ['OPENROUTER_API_KEY'])
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
pinecone_index = pc.Index("consultaddhackathonchatbot")

<ipython-input-4-fecd913df006>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
index_name = "consultaddhackathonchatbot"
namespace = "pdfdata1"

In [6]:
# Step 5: Upload a PDF file
from google.colab import files
uploaded = files.upload()

pdf_path = next(iter(uploaded))

Saving ELIGIBLE RFP - 1.pdf to ELIGIBLE RFP - 1.pdf


In [7]:
# Step 6: Extract text using pdfplumber
text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n"

In [8]:
# Step 7: Convert text to Langchain Document format
doc = Document(page_content=text, metadata={"source": pdf_path})

In [9]:
# Step 8: Token length function using tiktoken
tokenizer = tiktoken.get_encoding('p50k_base')
def tiktoken_len(text):
    return len(tokenizer.encode(text, disallowed_special=()))

In [10]:
# Step 9: Split text into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
    length_function=tiktoken_len
)
documents = splitter.split_documents([doc])

In [11]:
# Step 10: Add documents to Pinecone
vectorstore = PineconeVectorStore.from_texts(
    [d.page_content for d in documents],
    hf_embeddings,
    index_name=index_name,
    namespace=namespace
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 05 Apr 2025 22:01:41 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '107', 'x-pinecone-request-id': '776175114429744949', 'x-envoy-upstream-service-time': '34', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 384 does not match the dimension of the index 1024","details":[]}


In [12]:
pc.delete_index("consultaddhackathonchatbot")

In [16]:
from pinecone import ServerlessSpec

pc.create_index(
    name="consultaddhackathonchatbot",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "consultaddhackathonchatbot",
    "metric": "cosine",
    "host": "consultaddhackathonchatbot-bv0dgta.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [17]:
# Step 10: Add documents to Pinecone
vectorstore = PineconeVectorStore.from_texts(
    [d.page_content for d in documents],
    hf_embeddings,
    index_name=index_name,
    namespace=namespace
)

In [18]:
# Step 11: RAG query function
def perform_rag(query):
    query_embedding = hf_embeddings.embed_query(query)
    top_matches = pinecone_index.query(
        vector=query_embedding,
        top_k=5,
        include_metadata=True,
        namespace=namespace
    )

    contexts = [match['metadata'].get("text", "") + "\n" + match.get("id", "") for match in top_matches['matches']]
    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[:10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    primer = """You are a knowledgeable assistant. Answer questions using only the provided PDF content. Be precise, clear, and use facts from the document."""

    res = openrouter_client.chat.completions.create(
        model="mistralai/mistral-nemo",
        messages=[
            {"role": "system", "content": primer},
            {"role": "user", "content": augmented_query}
        ]
    )

    return res.choices[0].message.content

In [19]:
# Step 12: Test it
query = "What is this PDF about?"
response = perform_rag(query)
print(response)

Based on the provided context, this PDF is a 'Request for Proposal' (RFP) document labeled "RFP 25-008 Temporary Staffing Services". It is an invitation from MHMR of Tarrant County (MHMR) to potential temporary staffing agencies to submit proposals for providing temporary employees on an as-needed, non-exclusive basis. The RFP outlines the scope of work, specifications, and requirements for the temporary staffing services needed by MHMR. It also includes instructions for submitting proposals, as well as mandatory submission forms and required documents. The primary aim of this RFP is to select a suitable temporary staffing agency based on the proposals received.
